# Caso de estudio - Supervivencia en el Titanic

# Extracción de características

Ahora trataremos parte muy importante del aprendizaje automático: la extracción de características cuantitativas a partir de los datos. Con este fin:
- Aprenderemos como las características pueden extraerse a partir de datos del mundo real.
- Veremos como extraer características numéricas a partir de datos textuales.
Además, repasaremos algunas herramientas básicas en scikit-learn que pueden utilizarse para realizar estas tareas.

## ¿Qué son características?

### Características numéricas

Recuerda que los datos en scikit-learn vienen en arrays de dos dimensiones con tamaño **n_samples** $\times$ **n_features**.

Anteriormente, vimos el dataset iris, que tienen 150 ejemplos y 4 características.

In [49]:
from sklearn.datasets import load_iris

iris = load_iris()
print(iris.data.shape)

(150, 4)


Las características son:
- Longitud de sépalo en cm
- Ancho de sépalo en cm
- Longitud de pétalo en cm
- Ancho de pétalo en cm

Las características numéricas como estas son directas: cada ejemplo contiene una lista de números con precisión real que se corresponden con las características.

### Características categóricas

¿Qué pasa si tenemos características categóricas?. Por ejemplo, imagina que disponemos del color de cada flor de iris: $color \in \{red, blue, purple\}$

Podrías estar tentado de usar algo así como i.e. *red=1, blue=2, purple=3*, pero, en general, **esto es una mala idea**. Los estimadores tienden a trabajar con la suposición de que las características numéricas se sitúan en una escala continua por lo que, en este ejemplo, 1 y 2 serían más parecidos que 1 y 3 y esto no tiene porque ser generalmente verdad.

De hecho, el ejemplo anterior es una subcategoría de las variables categóricas, en concreto, una variable nominal. Las variables nominales no tienen asociado un orden, mientras que las variables ordinales si que implican un orden. Por ejemplo, las tallas de las camisetas formarían una variable ordinal "XL > L > M > S".

Una forma de transformar variables nominales en un formato que prevenga al estimador de asumir un orden es la llamada representación $1$-$de$-$J$ (*one-hot encoding*). Cada categoría genera su propia variable por separado.

El conjunto de características aumentado sería:
- Longitud de sépalo en cm
- Ancho de sépalo en cm
- Longitud de pétalo en cm
- Ancho de pétalo en cm
- color=purple (1.0 o 0.0)
- color=blue (1.0 o 0.0)
- color=red (1.0 o 0.0)

Observa que al usar este conjunto de características puede que los datos se representen mejor usando **matrices dispersas**, como veremos en el ejemplo de clasificación de texto que analizaremos después.

#### Utilizando DictVectorizer para codificar variables categóricas

Cuando los datos de entrada están codificados con un diccionario de tal forma que los valores son o cadenas o valores numéricos, se puede usar la clase `DictVectorizer` para obtener la expansión booleana sin tocar las características numéricas:

In [50]:
measurements = [
    {'city': 'Dubai', 'temperature': 33.},
    {'city': 'London', 'temperature': 12.},
    {'city': 'San Francisco', 'temperature': 18.}
]

In [51]:
from sklearn.feature_extraction import DictVectorizer

vec = DictVectorizer()
vec

DictVectorizer()

In [52]:
dataVec=vec.fit_transform(measurements).toarray()

In [53]:
# vec.get_feature_names_out()
columnas = vec.get_feature_names_out()
print(columnas)


['city=Dubai' 'city=London' 'city=San Francisco' 'temperature']


### Características derivadas

Otro tipo bastante común de características son las **características derivadas**, que son características obtenidas a partir de algún paso previo de preprocesamiento y que se supone que son más informativas que las originales. Este tipo de características pueden estar basadas en **extracción de características** y en **reducción de la dimensionalidad** (tales como PCA o aprendizaje de variedades) y pueden ser combinaciones lineales o no lineales de las características originales (como en regresión polinómica) o transformaciones más sofisticadas de las características.

### Combinando características numéricas y categóricas

Como un ejemplo de la forma en que se trabaja con datos numéricos y categóricos, vamos a realizar un ejercicio en el que predeciremos la supervivencia de los pasajeros del HMS Titanic.

Utilizaremos una versión del dataset Titanic que puede descargarse de [titanic3.xls](http://biostat.mc.vanderbilt.edu/wiki/pub/Main/DataSets/titanic3.xls). Previamente, ya hemos convertido el `.xls` a `.csv` para que sea más fácil su manipulación (como texto), de manera que los datos no fueron modificados.

Necesitamos leer todas las líneas del fichero `titanic3.csv`, ignorar la cabecera y encontrar las etiquetas (sobrevivió o murió) y los datos de entrada (características de la persona). Vamos a ver la cabecera y algunas líneas de ejemplo:

In [54]:
import os
import pandas as pd
import numpy as np
# Supongamos que 'fare' es una de las columnas con comas
titanic = pd.read_csv(
    os.path.join('datasets', 'titanic3.csv'),
    converters={
        'fare': lambda x: float(x.replace(',', '.')) if x not in ('', None) else np.nan,
        'age': lambda x: float(x.replace(',', '.')) if x not in ('', None) else np.nan
    }
)

#titanic = pd.read_csv(os.path.join('datasets', 'titanic3.csv'))
print(titanic.columns)

Index(['pclass', 'survived', 'name', 'sex', 'age', 'sibsp', 'parch', 'ticket',
       'fare', 'cabin', 'embarked', 'boat', 'body', 'home.dest'],
      dtype='object')


Aquí tenemos una descripción de lo que significan cada una de las variables:

```
pclass          Passenger Class
                (1 = 1st; 2 = 2nd; 3 = 3rd)
survival        Survival
                (0 = No; 1 = Yes)
name            Name
sex             Sex
age             Age
sibsp           Number of Siblings/Spouses Aboard
parch           Number of Parents/Children Aboard
ticket          Ticket Number
fare            Passenger Fare
cabin           Cabin
embarked        Port of Embarkation
                (C = Cherbourg; Q = Queenstown; S = Southampton)
boat            Lifeboat
body            Body Identification Number
home.dest       Home/Destination
```

Parece que las variables `name`, `sex`, `cabin`, `embarked`, `boat`, `body` y `homedest` son candidatas a ser variables categóricas, mientras que el resto parecen variables numéricas. Vamos a ver las primeras filas para tener un mejor conocimiento de la base de datos:

In [55]:
titanic.head()

,pclass,survived,name,sex,age,sibsp,parch,ticket,fare,cabin,embarked,boat,body,home.dest
0,1,1,"Allen, Miss. Elisabeth Walton",female,29.0000,0,0,24160,211.3375,B5,S,2,NaN,"St Louis, MO"
1,1,1,"Allison, Master. Hudson Trevor",male,0.9167,1,2,113781,151.5500,C22 C26,S,11,NaN,"Montreal, PQ / Chesterville, ON"
2,1,0,"Allison, Miss. Helen Loraine",female,2.0000,1,2,113781,151.5500,C22 C26,S,NaN,NaN,"Montreal, PQ / Chesterville, ON"
3,1,0,"Allison, Mr. Hudson Joshua Creighton",male,30.0000,1,2,113781,151.5500,C22 C26,S,NaN,135.0,"Montreal, PQ / Chesterville, ON"
4,1,0,"Allison, Mrs. Hudson J C (Bessie Waldo Daniels)",female,25.0000,1,2,113781,151.5500,C22 C26,S,NaN,NaN,"Montreal, PQ / Chesterville, ON"


In [56]:
titanic.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1309 entries, 0 to 1308
Data columns (total 14 columns):
 #   Column     Non-Null Count  Dtype  
---  ------     --------------  -----  
 0   pclass     1309 non-null   int64  
 1   survived   1309 non-null   int64  
 2   name       1309 non-null   object 
 3   sex        1309 non-null   object 
 4   age        1046 non-null   float64
 5   sibsp      1309 non-null   int64  
 6   parch      1309 non-null   int64  
 7   ticket     1309 non-null   object 
 8   fare       1308 non-null   float64
 9   cabin      295 non-null    object 
 10  embarked   1307 non-null   object 
 11  boat       486 non-null    object 
 12  body       121 non-null    float64
 13  home.dest  745 non-null    object 
dtypes: float64(3), int64(4), object(7)
memory usage: 143.3+ KB


In [57]:
titanic.select_dtypes(include=['object']).columns

Index(['name', 'sex', 'ticket', 'cabin', 'embarked', 'boat', 'home.dest'], dtype='object')

In [58]:
titanic.describe()

,pclass,survived,age,sibsp,parch,fare,body
count,1309.000000,1309.000000,1046.000000,1309.000000,1309.000000,1308.000000,121.000000
mean,2.294882,0.381971,29.881135,0.498854,0.385027,33.295479,160.809917
std,0.837836,0.486055,14.413500,1.041658,0.865560,51.758668,97.696922
min,1.000000,0.000000,0.166700,0.000000,0.000000,0.000000,1.000000
25%,2.000000,0.000000,21.000000,0.000000,0.000000,7.895800,72.000000
50%,3.000000,0.000000,28.000000,0.000000,0.000000,14.454200,155.000000
75%,3.000000,1.000000,39.000000,1.000000,0.000000,31.275000,256.000000
max,3.000000,1.000000,80.000000,8.000000,9.000000,512.329200,328.000000


In [59]:
titanic['pclass'].nunique()

3

In [60]:
titanic.describe(include=['object', 'string', 'category'])


,name,sex,ticket,cabin,embarked,boat,home.dest
count,1309,1309,1309,295,1307,486,745
unique,1307,2,929,186,3,27,369
top,"Kelly, Mr. James",male,CA. 2343,C23 C25 C27,S,13,"New York, NY"
freq,2,843,11,6,914,39,64


Podemos descartar directamente las columnas "boat" y "body" ya que está directamente relacionadas con que el pasajero sobreviviese. El nombre es (probablemente) único para cada persona y por tanto no es informativo. Vamos a intentar en primer lugar usar "pclass", "sibsp", "parch", "fare" y "embarked" como características:

In [61]:
labels = titanic.survived.values
features = titanic[['pclass', 'sex', 'age', 'sibsp', 'parch', 'fare', 'embarked']]

In [62]:
features.head()

,pclass,sex,age,sibsp,parch,fare,embarked
0,1,female,29.0000,0,0,211.3375,S
1,1,male,0.9167,1,2,151.5500,S
2,1,female,2.0000,1,2,151.5500,S
3,1,male,30.0000,1,2,151.5500,S
4,1,female,25.0000,1,2,151.5500,S


En principio, los datos ahora solo contienen características útiles, pero no están en un formato que los algoritmos de aprendizaje automático puedan entender. Necesitamos transformar las cadenas "male" y "female" en variables binarias que indiquen el género y lo mismo para `embarked`.Podemos hacer esto usando la función ``get_dummies`` de pandas:

In [63]:
pd.get_dummies(features).head()

,pclass,age,sibsp,parch,fare,sex_female,sex_male,embarked_C,embarked_Q,embarked_S
0,1,29.0000,0,0,211.3375,True,False,False,False,True
1,1,0.9167,1,2,151.5500,False,True,False,False,True
2,1,2.0000,1,2,151.5500,True,False,False,False,True
3,1,30.0000,1,2,151.5500,False,True,False,False,True
4,1,25.0000,1,2,151.5500,True,False,False,False,True


Esta transformación ha codificado bien las columnas de cadenas. Sin embargo, parece que la variable ``pclass`` también es una variable categórica. Podemos listar de forma explícita las variables que queremos codificar utilizando el parámetro ``columns`` para incluir ``pclass``:

In [64]:
features_dummies = pd.get_dummies(features, columns=['pclass', 'sex', 'embarked'])
# features_dummies = pd.get_dummies(features, columns=['pclass', 'sex', 'embarked'],drop_first=True)
features_dummies.head(n=16)

,age,sibsp,parch,fare,pclass_1,pclass_2,pclass_3,sex_female,sex_male,embarked_C,embarked_Q,embarked_S
0,29.0000,0,0,211.3375,True,False,False,True,False,False,False,True
1,0.9167,1,2,151.5500,True,False,False,False,True,False,False,True
2,2.0000,1,2,151.5500,True,False,False,True,False,False,False,True
3,30.0000,1,2,151.5500,True,False,False,False,True,False,False,True
4,25.0000,1,2,151.5500,True,False,False,True,False,False,False,True
5,48.0000,0,0,26.5500,True,False,False,False,True,False,False,True
6,63.0000,1,0,77.9583,True,False,False,True,False,False,False,True
7,39.0000,0,0,0.0000,True,False,False,False,True,False,False,True
8,53.0000,2,0,51.4792,True,False,False,True,False,False,False,True
9,71.0000,0,0,49.5042,True,False,False,False,True,True,False,False


In [65]:
#También podríamos hacerlo con DictVectorizer
from sklearn.feature_extraction import DictVectorizer

diccionario = features.to_dict('records')
vec = DictVectorizer()
dataset = vec.fit_transform(diccionario)
print(dataset.todense())

[[29.      0.      0.     ...  1.      0.      0.    ]
 [ 0.9167  0.      0.     ...  0.      1.      1.    ]
 [ 2.      0.      0.     ...  1.      0.      1.    ]
 ...
 [26.5     0.      1.     ...  0.      1.      0.    ]
 [27.      0.      1.     ...  0.      1.      0.    ]
 [29.      0.      0.     ...  0.      1.      0.    ]]


In [66]:
# Para obtener los nombres de las columnas
columnas = vec.get_feature_names_out()
print(columnas)

['age' 'embarked' 'embarked=C' 'embarked=Q' 'embarked=S' 'fare' 'parch'
 'pclass' 'sex=female' 'sex=male' 'sibsp']


La opción más actual y recomendada en scikit-learn para sustituir DictVectorizer es usar ColumnTransformer + OneHotEncoder, normalmente dentro de un Pipeline

In [67]:
from sklearn.compose import ColumnTransformer
from sklearn.preprocessing import OneHotEncoder
from sklearn.pipeline import Pipeline
from sklearn.impute import SimpleImputer

num_features = ['age', 'fare', 'sibsp', 'parch']
cat_features = ['pclass', 'sex', 'embarked']


In [68]:
numeric_transformer = Pipeline(steps=[
    ('imputer', SimpleImputer(strategy='median'))
])

categorical_transformer = Pipeline(steps=[
    ('imputer', SimpleImputer(strategy='most_frequent')),
    ('onehot', OneHotEncoder(handle_unknown='ignore'))
])


In [69]:
preprocessor = ColumnTransformer(
    transformers=[
        ('num', numeric_transformer, num_features),
        ('cat', categorical_transformer, cat_features)
    ]
)


In [70]:
from sklearn.dummy import DummyClassifier
from sklearn.model_selection import train_test_split

X = features
y = labels

X_train, X_test, y_train, y_test = train_test_split(
    X, y, test_size=0.3, random_state=0
)

model = Pipeline(steps=[
    ('preprocessor', preprocessor),
    ('classifier', DummyClassifier(strategy='most_frequent'))
])

model.fit(X_train, y_train)
print("Accuracy:", model.score(X_test, y_test))


Accuracy: 0.6259541984732825


In [71]:

data = features_dummies.values.astype(float)
print(data)

[[29.      0.      0.     ...  0.      0.      1.    ]
 [ 0.9167  1.      2.     ...  0.      0.      1.    ]
 [ 2.      1.      2.     ...  0.      0.      1.    ]
 ...
 [26.5     0.      0.     ...  1.      0.      0.    ]
 [27.      0.      0.     ...  1.      0.      0.    ]
 [29.      0.      0.     ...  0.      0.      1.    ]]


In [72]:
# Comprobamos que hay valores perdidos, tendremos que aplicar un Imputer
import numpy as np
np.isnan(data).any()
#features_dummies.isna().any().any()  # True si hay algún NaN en todo el DataFrame


np.True_

Una vez hemos hecho el trabajo de duro de cargar los datos, evaluar un clasificador con estos datos es directo. Vamos a ver que rendimiento obtenemos con el clasificador más simple, `DummyClassifier('most_frequent')`, que es equivalente al `ZeroR`.

In [73]:
from sklearn.model_selection import train_test_split
from sklearn.impute import SimpleImputer


train_data, test_data, train_labels, test_labels = train_test_split(
    data, labels,  test_size=0.3,random_state=0)

imp = SimpleImputer()
imp.fit(train_data)
train_data_finite = imp.transform(train_data)
test_data_finite = imp.transform(test_data)

In [74]:
np.isnan(train_data_finite).any()

np.False_

In [75]:
from sklearn.dummy import DummyClassifier

clf = DummyClassifier(strategy='most_frequent')
clf.fit(train_data_finite, train_labels)
print("Accuracy: %f"
      % clf.score(test_data_finite, test_labels))

Accuracy: 0.625954


<div class="alert alert-success">
    <b>EJERCICIO</b>:
     <ul>
      <li>
      Intenta ejecutar el problema de clasificación anterior pero usando ``LogisticRegression`` y ``RandomForestClassifier`` en lugar de ``DummyClassifier``
      </li>
      <li>
      Prueba a cambiar el conjunto de características considerado. ¿Consigues mejorar los resultados?
      </li>
    </ul>
</div>